# Dense Document Retriever - Implantação

Ranqueia textos de acordo com um outro texto de entrada através do ranqueador DPR do hugginfaces.<br>
### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [1]:
%%writefile Model.py
import json
import joblib
import pandas as pd
import numpy as np
from aux_functions import build_result_dataframe



class Model:
    
    def __init__(self):
        
        artifacts = joblib.load("/tmp/data/sparse_retriever.joblib")
        self.model = artifacts["model"]
        self.question = artifacts["question"]
        self.report_contexts = artifacts["report_contexts"]
        self.top = artifacts["top"]
        self.inner_batch_size = artifacts["inner_batch_size"]
        self.columns = artifacts["columns"]
        self.column_context = artifacts["column_context"]
        self.column_score = artifacts["column_score"]
        self.column_doc_id = artifacts["column_doc_id"] 

    def predict(self, X, feature_names, meta=None):
        
        if not feature_names:
            feature_names = self.columns
            
        df = pd.DataFrame(X,columns = feature_names)
        report_contexts = list(df[self.column_context])
        
        sim_contexts_ids, scores = self.model(questions=self.question,
                                     passages = report_contexts,
                                     inner_batch_size = self.inner_batch_size,
                                     top = self.top)
        
        
        df_result = build_result_dataframe(df_input=df,
                            sim_contexts_ids=sim_contexts_ids[0],
                            scores=scores[0],
                            column_doc_id = self.column_doc_id,
                            column_score = self.column_score)
        
        
        return df.to_numpy() 

Overwriting Model.py


In [2]:
import pandas as pd
df = pd.read_csv("/tmp/data/fabc-step3-4.csv")
X = df.to_numpy()

In [3]:
from Model import Model
model = Model()
result = model.predict(X,None)
result

array([[0,
        'HB19PR_SO06_SO07_SO08_SP_SO09 - Eficacia do herbicida imazethapyr associado a um inibidor da PROTOX para manejo de plantas daninhas na pre-emergencia da soja tolerante a glyphosate.pdf',
        '_raiz',
        'Setor de Herbologia HERBICIDAS RESIDUAIS NA CULTURA DA SOJA Eficácia do herbicida imazethapyr associado a um inibidor da PROTOX para manejo de plantas daninhas na pré-emergência da soja tolerante a glyphosate Autor: Luís Henrique Penckowski Eliana Fernandes Borsato Evandro H. G. Maschietto AVISO LEGAL Este documento está protegido por direitos autorais e pode conter informações confidenciais e/ou privilegiadas. É expressamente proibido copiar, modificar, distribuir, remover, adicionar ou divulgar o seu conteúdo, ou parte deste, em qualquer meio, sem o consentimento expresso e por escrito da FUNDAÇÃO ABC. Qualquer utilização das informações/dados de forma diversa do contido no presente documento afeta a precisão dos resultados e não reflete as conclusões da 